# Library Imports

In [23]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import os
from tqdm import tqdm

# Produce the plots for all datasets
We will produce the plots for all graphs, all participants, and two fingers for each participant

In [24]:
# Get the list of all file names in the directory
directory = './'  # Replace with the path to your directory
file_names = os.listdir(directory)
output_directory = './Plots/'

# Make the output directory in case it does not exist
if not os.path.exists(output_directory):
        os.makedirs(output_directory)
        print(f"Directory '{output_directory}' created successfully.")
else:
    print(f"Directory '{output_directory}' already exists.")

# Set the lists to iterate over
participant_numbers = [1, 2, 3, 4, 5] # 5 participants
finger_numbers = [0, 1] # Two fingers
graph_names = ['_histo_15_normal', '_histo_15_spatial',
               '_histo_training_normal', '_histo_training_spatial',
               '_scatter_100_normal', '_scatter_100_spatial', 
               '_scatter_training_normal', '_scatter_training_spatial']
file_extension = '.csv'

# Set the font sizes for the plots
fontsize_title = 20
fontsize_axis = 15
fontsize_ticks = 15
fontsize_legend = 15

# Set the max number of pixels in each axis for your tablet
maxX = 2560 # Pixels
maxY = 1600 # Pixels

for participant_number in tqdm(participant_numbers):
    for graph_name in graph_names:
        for finger in finger_numbers:
            # Obtain the file name
            file_name = str(participant_number) + graph_name + file_extension
            
            # Try to find the file and produce the plots
            try:
                # Import the dataframe
                df = pd.read_csv(file_name, sep = ",")
                # Only keep rows where there are fingers on screen!
                # Drop rows where values in Columns X and Y are zero
                df_filtered = df.drop(df.loc[(df['x0'] == 0) &
                                             (df['x1'] == 0) &
                                             (df['x2'] == 0) &
                                             (df['x3'] == 0) &
                                             (df['x4'] == 0) &
                                             (df['y0'] == 0) &
                                             (df['y1'] == 0) &
                                             (df['y2'] == 0) &
                                             (df['y3'] == 0) &
                                             (df['y4'] == 0)].index)

                # Reset the index of the filtered DataFrame
                df_filtered.reset_index(drop=True, inplace=True)

                ########################################################################
                # Produce the Time-wise Scatterplot
                ########################################################################
                # Clear any previous figure
                plt.clf()
                
                # Set the min and max for this colorbar
                order = np.linspace(0, 100, len(df_filtered))                
                
                # Set the plot background color
                ax = plt.axes()
                # Setting the background color of the plot
                ax.set_facecolor('#440154')

                # Create the scatter plot
                plt.scatter(df_filtered['x' + str(finger)], df_filtered['y' + str(finger)], c=order, cmap='viridis')
                plt.gca().invert_yaxis()
                # Get the current X-axis object
                x_axis = plt.gca().xaxis
                
                # Move the X-axis to the top
                x_axis.set_ticks_position('top')
                x_axis.set_label_position('top')

                # Set the title of the plot
                title_header = "Exploration Strategy"
                if "scatter" in file_name.replace('.csv', ""):
                    title_plot_type = "Scatter Plot"
                elif "histo" in file_name.replace('.csv', ""):
                    title_plot_type = "Histogram"
                if "normal" in file_name.replace('.csv', ""):
                    title_audio_type = "Normal Audio"
                elif "spatial" in file_name.replace('.csv', ""):
                    title_audio_type = "Spatial Audio"
                title_participant_number = file_name.replace('.csv', "")[0] # The first character of the string
                title_finger_number = str(finger + 1)
                plot_title = (title_header + ": " + 
                              title_plot_type + ", " +
                              title_audio_type + ", " + 
                              "Participant " + str(title_participant_number) + ", " +
                              "Finger " + str(title_finger_number))
#                 plt.title("Exploration Strategy: " + file_name.replace('.csv', "") + "_f" + str(finger), size = fontsize_title)
                plt.title(plot_title, size = fontsize_title)
                
                # Set the name and font size of the X axis
                plt.xlabel("X [px]", size = fontsize_axis)
                plt.xticks(fontsize = fontsize_ticks)
                
                # Set the name and font size of the Y axis
                plt.ylabel("Y [px]", size = fontsize_axis)
                plt.yticks(fontsize = fontsize_ticks)
                
                # Add a colorbar with labels and set its title and font size
                cbar = plt.colorbar(shrink = 1.0)
                cbar.set_label('Exploration Time %', fontsize = fontsize_legend)
                cbar.ax.tick_params(labelsize=fontsize_ticks) 
                
                # Set the figure size in inches, keeping the same ratio as the real-life table
                fig = plt.gcf()
                base_size = 10 # inches
                fig.set_size_inches(base_size, base_size*maxY/maxX)
                                
                # Show the plot
                fig.savefig(output_directory + file_name.replace('.csv', '') + 
                            '_scatter_f' + str(finger), 
                           bbox_inches='tight')
                
                ########################################################################
                # Produce the Finger Location 2D Histogram
                ########################################################################
                # Clear any previous figure
                plt.clf()

                # Set the size of each grid cell in pixels
                bin_size = 44 # Pixels

                # Plot the 2D histogram
                plt.hist2d(df_filtered['x' + str(finger)], df_filtered['y' + str(finger)], 
                           bins=[np.arange(0, maxX, bin_size), np.arange(0, maxY, bin_size)], 
                           cmap='viridis')

                # Invert the Y axis to match the origin of the Android tablet (the origin is top left)
                plt.gca().invert_yaxis()

                # Put X-axis on top
                ax = plt.gca()
                ax.xaxis.tick_top()
                ax.xaxis.set_label_position('top')

                # Change the axes aspect ratio if needed
                # ax.set_aspect(maxY/maxX)

                # Set the title of the plot
                title_header = "Point Density Map"
                plot_title = (title_header + ": " + 
                              title_plot_type + ", " +
                              title_audio_type + ", " + 
                              "Participant " + str(title_participant_number) + ", " +
                              "Finger " + str(title_finger_number))
                plt.title(plot_title, size = fontsize_title)
                
                # Set the name and font size of the X axis
                plt.xlabel("X [px]", size = fontsize_axis)
                plt.xticks(fontsize = fontsize_ticks)

                # Set the name and font size of the Y axis
                plt.ylabel("Y [px]", size = fontsize_axis)
                plt.yticks(fontsize = fontsize_ticks)

                # Add a colorbar with labels and set its title and font size
                cbar = plt.colorbar(shrink = 1.0)
                cbar.set_label('Point Density', fontsize = fontsize_legend)
                cbar.ax.tick_params(labelsize=fontsize_ticks) 

                # Set the figure size in inches, keeping the same ratio as the real-life table
                fig = plt.gcf()
                base_size = 10 # inches
                fig.set_size_inches(base_size, base_size*maxY/maxX)

                # Show the plot
                fig.savefig(output_directory + file_name.replace('.csv', '') + '_2Dhist_f' + str(finger), 
                           bbox_inches='tight')

            # ELSE
            except Exception:
#                 print("Error when processing file: " + file_name)
                pass

Directory './Plots/' already exists.


100%|█████████████████████████████████████████████| 5/5 [00:50<00:00, 10.03s/it]
